MCMC Probabilistic Matrix Factorization
=========

Adapted very lightly from: https://stackoverflow.com/questions/29736966/bayesian-probabilistic-matrix-factorization-bpmf-with-pymc3-positivedefinitee

Which is, itself, I believe based on "Bayesian Probabilistic Matrix Factorization using Markov Chain Monte Carlo", Ruslan Salakhutdinov and Andriy Mnih, in _Proceedings of the 25th International Conference on Machine Learning_, Helsinki, Finland, 2008.


In [16]:
import warnings
warnings.filterwarnings('ignore', '.*ix is deprecated.*')

In [32]:
import pymc3 as pm
import numpy as np
import pandas as pd
import theano
import scipy as sp

data = pd.read_table('test/sample_abundances.tsv', index_col=0)
strain_abunds = pd.read_table('test/strain_abundances.tsv', index_col=0)
strain_snps = pd.read_table('test/strain_genotypes.tsv', index_col=0)
n, m = data.shape
print(strain_abunds.shape)
print(strain_snps.shape)
print(data.shape)
print(np.abs(data - np.matmul(strain_snps, strain_abunds.T)).mean().mean())


(96, 63)
(10000, 63)
(10000, 96)
2.6131115500593887e-18


In [18]:
test_size = m // 10
train_size = m - test_size

print(n, m, test_size, train_size)
train = data.copy()
train.ix[:,train_size:] = np.nan  # remove test set data
train[train.isnull()] = train.mean().mean()  # mean value imputation
train = train.values

test = data.copy()
test.ix[:,:train_size] = np.nan  # remove train set data
test = test.values    

# Low precision reflects uncertainty; prevents overfitting
alpha_u = alpha_v = 1/np.var(train)
alpha = np.ones((n,m)) * 2  # fixed precision for likelihood function
dim = 63  # dimensionality

# Specify the model.
with pm.Model() as pmf:
    pmf_U = pm.MvNormal('U', mu=0, tau=alpha_u * np.eye(dim),
                        shape=(n, dim), testval=np.random.randn(n, dim)*.01)
    pmf_V = pm.MvNormal('V', mu=0, tau=alpha_v * np.eye(dim),
                        shape=(m, dim), testval=np.random.randn(m, dim)*.01)
    pmf_R = pm.Normal('R', mu=theano.tensor.dot(pmf_U, pmf_V.T),
                      tau=alpha, observed=train)

    # Find mode of posterior using optimization
    start = pm.find_MAP(method='powell')

10000 96 9 87


/opt/sys/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
/opt/sys/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()
  0%|          | 0/5000 [00:00<?, ?it/s]/opt/sys/anaconda3/lib/python3.6/site-packages/scipy/optimize/_minimize.py:502: RuntimeWarning: Method powell does not use gradient information (jac).
  RuntimeWarning)
logp = -2.4966e+05, ||grad|| = 11,029: : 5001it [01:21, 61.17it/s]                        



In [25]:
n, m = data.shape
dim = 10  # dimensionality
beta_0 = 1  # scaling factor for lambdas; unclear on its use
alpha = np.ones((n,m)) * 2  # fixed precision for likelihood function

print('building the BPMF model')
std = .05  # how much noise to use for model initialization
with pm.Model() as bpmf:
    # Specify user feature matrix
    lambda_u = pm.LKJCorr(
        'lambda_u', n=dim, V=np.eye(dim), eta=(dim, dim),
        testval=np.random.randn(dim, dim) * std)
    mu_u = pm.Normal(
        'mu_u', mu=0, tau=beta_0 * lambda_u, shape=dim,
        testval=np.random.randn(dim) * std)
    U = pm.MvNormal(
        'U', mu=mu_u, tau=lambda_u, shape=(n, dim),
        testval=np.random.randn(n, dim) * std)

    # Specify item feature matrix
    lambda_v = pm.Wishart(
        'lambda_v', n=dim, V=np.eye(dim), shape=(dim, dim),
        testval=np.random.randn(dim, dim) * std)
    mu_v = pm.Normal(
        'mu_v', mu=0, tau=beta_0 * lambda_v, shape=dim,
         testval=np.random.randn(dim) * std)
    V = pm.MvNormal(
        'V', mu=mu_v, tau=lambda_v, shape=(m, dim),
        testval=np.random.randn(m, dim) * std)

    # Specify rating likelihood function
    R = pm.Normal(
        'R', mu=theano.tensor.dot(U, V.T), tau=alpha,
        observed=train)

# `start` is the start dictionary obtained from running find_MAP for PMF.
for key in bpmf.test_point:
    if key not in start:
        start[key] = bpmf.test_point[key]


building the BPMF model


TypeError: __init__() got an unexpected keyword argument 'V'

In [38]:
n, m = data.shape
dim = 10  # dimensionality
beta_0 = 1  # scaling factor for lambdas; unclear on its use
alpha = np.ones((n,m)) * 2  # fixed precision for likelihood function

print('building the BPMF model')
std = .05  # how much noise to use for model initialization
with pm.Model() as bpmf:
    # Specify user feature matrix
    lambda_u = pm.Wishart(
        'lambda_u', nu=dim, V=np.eye(dim), shape=(dim, dim),
        testval=np.random.randn(dim, dim) * std)
    mu_u = pm.Normal(
        'mu_u', mu=0, tau=beta_0 * lambda_u, shape=dim,
        testval=np.random.randn(dim) * std)
    U = pm.MvNormal(
        'U', mu=mu_u, tau=lambda_u, shape=(n, dim),
        testval=np.random.randn(n, dim) * std)

    # Specify item feature matrix
    lambda_v = pm.Wishart(
        'lambda_v', nu=dim, V=np.eye(dim), shape=(dim, dim),
        testval=np.random.randn(dim, dim) * std)
    mu_v = pm.Normal(
        'mu_v', mu=0, tau=beta_0 * lambda_v, shape=dim,
         testval=np.random.randn(dim) * std)
    V = pm.MvNormal(
        'V', mu=mu_v, tau=lambda_v, shape=(m, dim),
        testval=np.random.randn(m, dim) * std)

    # Specify rating likelihood function
    R = pm.Normal(
        'R', mu=theano.tensor.dot(U, V.T), tau=alpha,
        observed=train)

# `start` is the start dictionary obtained from running find_MAP for PMF.
for key in bpmf.test_point:
    if key not in start:
        start[key] = bpmf.test_point[key]

with bpmf:
    step = pm.NUTS(scaling=start)

building the BPMF model


/home/pcombs/.local/lib/python3.6/site-packages/pymc3/distributions/multivariate.py:694: UserWarning: The Wishart distribution can currently not be used for MCMC sampling. The probability of sampling a symmetric matrix is basically zero. Instead, please use LKJCholeskyCov or LKJCorr. For more information on the issues surrounding the Wishart see here: https://github.com/pymc-devs/pymc3/issues/538.
  UserWarning)
/home/pcombs/.local/lib/python3.6/site-packages/pymc3/distributions/multivariate.py:694: UserWarning: The Wishart distribution can currently not be used for MCMC sampling. The probability of sampling a symmetric matrix is basically zero. Instead, please use LKJCholeskyCov or LKJCorr. For more information on the issues surrounding the Wishart see here: https://github.com/pymc-devs/pymc3/issues/538.
  UserWarning)


LinAlgError: Singular matrix

In [ ]:
pm.LKJCorr?